# Tensorflow 物件辨識 API 自動化流程

你只要一步一步來，就可以做出一個屬於你的模型！

* 使用 Jupyter 筆記本
* 參考資料：https://txz44wy3qbnec67omxio6h3gfm-ac4c6men2g7xr2a-readthedocs.translate.goog/en/latest/training.html#preparing-the-workspace

> ### 注意
> 請使用 VSCode + Jupyter

## 流程

流程共分為3個步驟：

1. 初始化
    1. 安裝、架設環境
    2. 設定環境
    3. 建立工作區
2. 準備資料集（dataset）
    1. 收集資料集（Dataset）
    2. Label 你的圖片
    3. 分類圖片
3. 訓練前準備（traing）
4. 執行訓練
5. 驗證和評估模型（model）
6. 導出模型（model）

## 1.初始化

### 1-1. 安裝、架設環境

如果你還沒有初始化`pipenv`，請先初始化一個環境，並且安裝上`ipykernal`


In [ ]:
! pipenv --three 3.7        # Python 3.7 是必要的喔！
! pipenv install ipykernal  # 這個是讓Jupyter可以在VSCode裡運作的package

啊對了，如果你是mac你用不了GPU加速喔！
因為我也是mac所以用不了，就不解釋了。 （www



### 1-2. 設定環境

1. 請使用 VSCode `Cmd+Shift+P`
2. 選擇：`Python:Select Interpreter`
3. 選擇剛剛創立的 `pipenv`

### 1-3. 建立工作區

然後 git clone tensorflow 的 model zoo 下來來使用 Object Detection API：

In [ ]:
! git clone git@github.com:tensorflow/models.git

接下來我們要建立工作環境，也就是一堆有規範的資料夾：

```
TensorFlow/
├─ addons/ (Optional)
│  └─ labelImg/
├─ models/
│  ├─ community/
│  ├─ official/
│  ├─ orbit/
│  ├─ research/
│  └─ ...
└─ workspace/
   └─ training_demo/  <---(專案資料夾)
```

專案資料夾結構:
```
training_demo/
├─ annotations/         <---(TFrecord和csv的儲存地)
├─ exported-models/     <---(模型的導出版本)
├─ images/              <---(dataset儲存地)
│  ├─ test/                 <--(測試組 資料集)
│  └─ train/                <--(訓練組 資料集)
├─ models/              <--(模型)
├─ pre-trained-models/  <--(預訓練模型)
└─ README.md
```

讓我們建立 workspace 資料夾吧！(如果有創立好了就不用建了)

In [ ]:
! mkdir TensorFlow
! mkdir TensorFlow/data
! mkdir TensorFlow/data/images
! mkdir TensorFlow/scripts
! mkdir TensorFlow/workspace
! mkdir TensorFlow/workspace/training_demo/
! mkdir TensorFlow/workspace/training_demo/annotations
! mkdir TensorFlow/workspace/training_demo/exported-models
! mkdir TensorFlow/workspace/training_demo/images
! mkdir TensorFlow/workspace/training_demo/images/test
! mkdir TensorFlow/workspace/training_demo/images/train
! mkdir TensorFlow/workspace/training_demo/models
! mkdir TensorFlow/workspace/training_demo/pre-trained-models

### 1-4. 最後最後

最後最後，把該 import 都 import：

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import random

print("=== finished")

## 2.準備數據集

首先把你的資料全部收集起來，然後放到資料夾`TensorFlow/data`中。

### 2-1.收集資料集（Dataset）

蒐集到你要的資料，全部蒐集起來。在這裏我有打包一個zip裡面全部都是img，然後放到data資料夾內。

In [ ]:
! python3 ./scripts/homebrew/generate_images.py

好了，圖片都準備就緒了，接下來label我們的圖片！

### 2-2.Label 你的圖片

執行的步驟如下：

1. 安裝 LabelImg
2. 啟動 LabelImg
3. Label Images

在這裡我們使用 LabelImg

安裝 LabelImg 工具，依照官網範例：https://pypi.org/project/labelImg/



In [ ]:
! pip3 install labelImg # 安裝 labelImg
! labelImg # 啟動 labelImg
# ! labelImg [IMAGE_PATH] [PRE-DEFINED CLASS FILE]

### 2-3.分類圖片

用預先寫好的分類器分類所有圖的 data set 和 label set, 程式會依照simple_rate比例分配test data到test和train資料夾中。



In [ ]:
# 使用ljcucc自製的：
! python3 ./scripts/homebrew/split_dataset.py 0.1 # 0.1 is the simple_rate

# 使用官方的 (不推薦))：
# ! python3 ./scripts/tutorial/partition_dataset.py --xml --imageDir [FULLPATH_TO_FOLDER] --ratio 0.1

### 2-4. 新建Label map

TensorFlow需要一個Label map映射，即將每個使用的Label映射到一個「**整數值**」。train和valid過程都使用此Label map。

下面，我們abel_map.pbtxt假設我們的dataset包含2個標籤，dogs並顯示了一個示例標籤圖（例如）cats：

```
item {
    id: 1
    name: 'cat'
}

item {
    id: 2
    name: 'dog'
}
```

當然我都幫你寫好了，從 `./assets` 中拉出 `label_map.pbtxt` 改一改即可